In [1]:
# Import Libraries
import requests
import json
import xlwt
import pandas as pd

In [2]:
# Function to store the data.
def store(i):
  case = {'url':i['prism:url'],
          'title':i['dc:title'],
          'author':i.get('dc:creator','NaN'),
          'publicationname':i.get('prism:publicationName','NaN'),
          'issn':i.get('prism:issn','NaN'),
          'eissn':i.get('prism:eIssn','NaN'),
          'volume':i.get('prism:volume','NaN'),
          'issue':i.get('prism:issueIdentifier','NaN'),
          'pagerange':i.get('prism:pageRange','NaN'),
          'coverdate':i.get('prism:coverDate','NaN'),
          'coverdisplaydate':i.get('prism:coverDisplayDate','NaN'),
          'doi':i.get('prism:doi','NaN'),
          'citedbycount':int(i.get('citedby-count','NaN')),
          'instName':(instName)
          }
  case_list.append(case)

In [3]:
# Import excel file of queries
df = pd.read_excel('QueryList.xlsx')

In [4]:
case_list = []
for a in df.itertuples():
  instName = a[1]
  instQuery = a[2]
  key = "xxxxxxxxxxxxxxxxxxxxxxx"
  # url = "https://api.elsevier.com/content/search/scopus?query="+instQuery+""
  url = "https://api.elsevier.com/content/search/scopus?query="+instQuery
  headers = {'Accept':'application/json','X-ELS-APIKey':key}
  r = requests.get(url, headers=headers)
  info = r.json()
  TotalResult = info['search-results']['opensearch:totalResults']

  for i in info['search-results']['entry']:
    store(i)
    las = int(info['search-results']['opensearch:startIndex']) + int(info['search-results']['opensearch:itemsPerPage'])
    
    while int(info['search-results']['opensearch:totalResults']) >= int(info['search-results']['opensearch:startIndex']) + int(info['search-results']['opensearch:itemsPerPage']):
      # url = "https://api.elsevier.com/content/search/scopus?query="+str(instQuery)+"&start="+str(las)+"&field=dc:title,dc:creator,prism:publicationName,prism:volume,prism:pageRange,prism:coverDate,prism:doi,citedby-count"
      url = "https://api.elsevier.com/content/search/scopus?query="+str(instQuery)+"&start="+str(las)
      r = requests.get(url, headers=headers)
      info = r.json()
      las = int(info['search-results']['opensearch:startIndex']) + int(info['search-results']['opensearch:itemsPerPage'])
      for i in info['search-results']['entry']:
        store(i)
  print(instName,'=>',TotalResult)

Hon. Shri. Annasaheb Dange College Of Engineering and Technology => 71
Baba Banda Singh Bahadur Engineering College => 114
INFLIBNET => 41


In [5]:
# Convert case_list to DataFrame and in Excel file
Metadata = pd.DataFrame(case_list)
Metadata.to_excel('Metadata.xlsx')
print('Metadata file created')

# Apply Count and Sum on data group by institute name 
Summary = Metadata.groupby('instName')['citedbycount'].agg(['count','sum'])

# Rename the field name
finalOut = Summary.reset_index().rename(columns={'instName':'Institute Name','count':'No of Publications','sum':'Total Citation'})

# Saving final output in Excel file
finalOut.to_excel('finalOut.xlsx')
finalOut

Metadata file created


,Institute Name,No of Publications,Total Citation
0,Baba Banda Singh Bahadur Engineering College,114,661
1,Hon. Shri. Annasaheb Dange College Of Engineer...,71,196
2,INFLIBNET,41,140
